# WeatherPy

## Imports

In [1]:
import numpy as np
import pandas as pd
import json 
import requests
import matplotlib.pyplot as plt
from citipy import citipy
from scipy import stats
from numpy.random import seed
from api_keys import weather_api_key
from api_keys import g_key


## Generate Cities List

In [2]:
#First want to create my list of 500 lat/lng pairs
# it says AT LEAST 500 so how many?????
# assuming best way is to obtain from a dataframe like we did in examples
# Lat and Long: in decimal degrees format and range from -90 to 90 for latitude and -180 to 180 for longitude.
# so for randomness will need to go through whole range of lats and whole range of longs
# also need to make sure they are unique pairs.
# could I make massive list and remove duplicates?? or is there a unique random function

# don't want integers for lat/long b/c need decimals
# numpy.random.uniform(low=0.0, high=1.0, size=None) seems best
# everything else only goes b/w 0 and 1 for numpy.random and other random features
# also seems you could use  import random   random.uniform(a, b)
# round(random.uniform(1,2), N) to ensure decimals

# also, how many decimal places do lat/long need. I saw somehting that said 15
# I wonder if there is a requirement for the google thingo and if I can round




# this sets it so that everytime I run seed(1) and then the random thingo it will give the same random numbers
# just to help me get a handle on what's happening
# works for both
# setting seed(1) so that although the numbers are random they staty the same each time I run my code.
seed(1)


# obtaining random samples from uniform distribution, so that each value is equally likely
# values are between the bounds for latitude and longitude, and will contain decimals

lat_number = np.random.uniform(-90, 90, 1300)

long_number = np.random.uniform(-180, 180, 1300)

#print(lat_number)
#print(long_number)

In [3]:
# make my data frame out of these lists

number_df = pd.DataFrame({"Latitude":lat_number, "Longitude": long_number})
number_df.head()

,Latitude,Longitude
0,-14.936039,13.322562
1,39.658409,-48.245995
2,-89.979413,37.550464
3,-35.580137,165.168906
4,-63.583940,-38.583740


In [4]:
#Now to use citipy
# will need to loop through the lats and longs find a city for each, and then put these cities into a list

# setting enmpty city and country lists
cities = []

country = []

# itterating through each row using the index
for x in number_df.index:
    lat = number_df['Latitude'][x]
    long = number_df['Longitude'][x]
    city = citipy.nearest_city(lat, long)
    city_name = city.city_name
    country_code = city.country_code
    
    cities.append(city_name)
    country.append(country_code)
    
# do I then just add them to my dataframe??

number_df["City"] = cities
number_df["Country"] = country
number_df

#>>> city = citipy.nearest_city(22.99, 120.21)
#>>> city
#<citipy.City instance at 0x1069b6518>
#>>>
#>>> city.city_name     # Tainan, my home town
#'tainan'
#>>>
#>>> city.country_code
#'tw'                  # And the country is surely Taiwan


#Randomly select at least 500 unique (non-repeat) cities based on latitude and longitude.
#The city data you generate is based on random coordinates as well as different query times; as such, 
#your outputs will not be an exact match to the provided starter notebook.

# what is a query time?
# i think it is time taken for requested info to return
# so I don't think it is something I need to worry about 

,Latitude,Longitude,City,Country
0,-14.936039,13.322562,lubango,ao
1,39.658409,-48.245995,torbay,ca
2,-89.979413,37.550464,port elizabeth,za
3,-35.580137,165.168906,ahipara,nz
4,-63.583940,-38.583740,ushuaia,ar
...,...,...,...,...
1295,56.959632,1.215939,boddam,gb
1296,-2.929149,9.777663,gamba,ga
1297,16.191234,-76.291348,morant bay,jm
1298,51.707421,4.319723,steenbergen,nl


In [5]:
# Checking for duplicate cities

duplicate_rows = number_df[number_df.duplicated(subset = ['City'])]
duplicate_rows
print(f"There are {len(number_df['City'])} duplicated rows that need to be removed from the DataFrame.")

There are 1300 duplicated rows that need to be removed from the DataFrame.


In [6]:
# dropping dupicate cities, keeping the first occuurance of each city (default)

number_df = number_df.drop_duplicates(subset=['City'])

# resetting the index and dropping the old index
number_df = number_df.reset_index(drop=True)
number_df


,Latitude,Longitude,City,Country
0,-14.936039,13.322562,lubango,ao
1,39.658409,-48.245995,torbay,ca
2,-89.979413,37.550464,port elizabeth,za
3,-35.580137,165.168906,ahipara,nz
4,-63.583940,-38.583740,ushuaia,ar
...,...,...,...,...
567,50.440424,-110.868744,medicine hat,ca
568,8.361826,124.960650,maluko,ph
569,56.959632,1.215939,boddam,gb
570,16.191234,-76.291348,morant bay,jm


In [7]:
# Creating new columns in the dataframe for the weather API information

cities_df = number_df.copy()

cities_df['Max Temperature (F)'] = ""
cities_df['Humidity'] = ""
cities_df['Cloudiness'] = ""
cities_df['Wind Speed (mph)'] = ""
cities_df['Date'] = ""

cities_df.head()


,Latitude,Longitude,City,Country,Max Temperature (F),Humidity,Cloudiness,Wind Speed (mph),Date
0,-14.936039,13.322562,lubango,ao,,,,,
1,39.658409,-48.245995,torbay,ca,,,,,
2,-89.979413,37.550464,port elizabeth,za,,,,,
3,-35.580137,165.168906,ahipara,nz,,,,,
4,-63.583940,-38.583740,ushuaia,ar,,,,,


In [8]:
# TEST
#doing json thing to see what it is like
#url = "http://api.openweathermap.org/data/2.5/weather?"
#units = "metric"

#above is base url

# Build partial query URL
#query_url = f"{url}appid={weather_api_key}&units={units}&q="
#city = 'melbourne'

#response = requests.get(query_url + city).json()

#print(json.dumps(response, indent = 4, sort_keys = True))

#print (response["dt"])

In [9]:
# setting up loop

# how do I do this in sets
# example seems to go in sets of 50 

# base url
url = "http://api.openweathermap.org/data/2.5/weather?"

# setting units to imperial for Fahrenheit
units = "imperial"


# Build partial query URL
query_url = f"{url}appid={weather_api_key}&units={units}&q="


 # assemble url and make API request
print("Beginning Data Retrieval")
print("-------------------------")

# use iterrows to iterate through pandas dataframe
for index, row in cities_df.iterrows():

    # get city from df
    city = row['City']
    
    # getting country from df
    country = row['Country']

    response = requests.get(query_url + city + "," + country).json()

    # adding to data frame using try and except incase no data
    
    try:
        print(f"Retrieving Results for Index {index}: {city}, {country}.")
        
        cities_df.loc[index, 'Max Temperature (F)'] = response["main"]["temp_max"]
        cities_df.loc[index, 'Humidity'] = response["main"]["humidity"]
        cities_df.loc[index, 'Cloudiness'] = response["clouds"]["all"]
        cities_df.loc[index, 'Wind Speed (mph)'] = response["wind"]["speed"]
        cities_df.loc[index, 'Date'] = response["dt"]
          
    except (KeyError, IndexError):
        print("City not found... skipping.")
        
    print("------------")
    


Beginning Data Retrieval
-------------------------
Retrieving Results for Index 0: lubango, ao.
------------
Retrieving Results for Index 1: torbay, ca.
------------
Retrieving Results for Index 2: port elizabeth, za.
------------
Retrieving Results for Index 3: ahipara, nz.
------------
Retrieving Results for Index 4: ushuaia, ar.
------------
Retrieving Results for Index 5: busselton, au.
------------
Retrieving Results for Index 6: taolanaro, mg.
City not found... skipping.
------------
Retrieving Results for Index 7: caravelas, br.
------------
Retrieving Results for Index 8: kulhudhuffushi, mv.
------------
Retrieving Results for Index 9: conde, br.
------------
Retrieving Results for Index 10: half moon bay, us.
------------
Retrieving Results for Index 11: arraial do cabo, br.
------------
Retrieving Results for Index 12: upernavik, gl.
------------
Retrieving Results for Index 13: punta arenas, cl.
------------
Retrieving Results for Index 14: hasaki, jp.
------------
Retrievin

Retrieving Results for Index 127: kamir, pk.
------------
Retrieving Results for Index 128: saldanha, za.
------------
Retrieving Results for Index 129: cianorte, br.
------------
Retrieving Results for Index 130: rock sound, bs.
------------
Retrieving Results for Index 131: severo-kurilsk, ru.
------------
Retrieving Results for Index 132: te anau, nz.
------------
Retrieving Results for Index 133: athabasca, ca.
------------
Retrieving Results for Index 134: tuatapere, nz.
------------
Retrieving Results for Index 135: litoral del san juan, co.
City not found... skipping.
------------
Retrieving Results for Index 136: vila franca do campo, pt.
------------
Retrieving Results for Index 137: cardston, ca.
------------
Retrieving Results for Index 138: cherskiy, ru.
------------
Retrieving Results for Index 139: klaksvik, fo.
------------
Retrieving Results for Index 140: taoudenni, ml.
------------
Retrieving Results for Index 141: bosaso, so.
------------
Retrieving Results for Index

Retrieving Results for Index 253: khuzhir, ru.
------------
Retrieving Results for Index 254: sidi ali, dz.
------------
Retrieving Results for Index 255: roros, no.
------------
Retrieving Results for Index 256: hithadhoo, mv.
------------
Retrieving Results for Index 257: vigrestad, no.
------------
Retrieving Results for Index 258: ponta do sol, cv.
------------
Retrieving Results for Index 259: barbalha, br.
------------
Retrieving Results for Index 260: lewiston, us.
------------
Retrieving Results for Index 261: appleton, us.
------------
Retrieving Results for Index 262: campoverde, pe.
------------
Retrieving Results for Index 263: alotau, pg.
City not found... skipping.
------------
Retrieving Results for Index 264: angoram, pg.
------------
Retrieving Results for Index 265: tshela, cd.
------------
Retrieving Results for Index 266: stoyba, ru.
City not found... skipping.
------------
Retrieving Results for Index 267: alto araguaia, br.
------------
Retrieving Results for Inde

Retrieving Results for Index 380: mazatlan, mx.
------------
Retrieving Results for Index 381: moose factory, ca.
------------
Retrieving Results for Index 382: iringa, tz.
------------
Retrieving Results for Index 383: kushima, jp.
------------
Retrieving Results for Index 384: paradwip, in.
City not found... skipping.
------------
Retrieving Results for Index 385: touros, br.
------------
Retrieving Results for Index 386: kommunisticheskiy, ru.
------------
Retrieving Results for Index 387: aklavik, ca.
------------
Retrieving Results for Index 388: timbiqui, co.
------------
Retrieving Results for Index 389: zheleznodorozhnyy, ru.
------------
Retrieving Results for Index 390: constantine, dz.
------------
Retrieving Results for Index 391: champerico, gt.
------------
Retrieving Results for Index 392: cockburn town, tc.
------------
Retrieving Results for Index 393: illoqqortoormiut, gl.
City not found... skipping.
------------
Retrieving Results for Index 394: high level, ca.
-----

Retrieving Results for Index 508: puksoozero, ru.
------------
Retrieving Results for Index 509: sur, om.
------------
Retrieving Results for Index 510: nong han, th.
------------
Retrieving Results for Index 511: issia, ci.
------------
Retrieving Results for Index 512: maunabo, us.
City not found... skipping.
------------
Retrieving Results for Index 513: kamenskoye, ru.
City not found... skipping.
------------
Retrieving Results for Index 514: margate, za.
------------
Retrieving Results for Index 515: louisbourg, ca.
City not found... skipping.
------------
Retrieving Results for Index 516: kasama, zm.
------------
Retrieving Results for Index 517: crab hill, bb.
City not found... skipping.
------------
Retrieving Results for Index 518: conceicao do araguaia, br.
------------
Retrieving Results for Index 519: bereda, so.
City not found... skipping.
------------
Retrieving Results for Index 520: saint-leu, re.
------------
Retrieving Results for Index 521: coahuayana, mx.
----------

In [12]:
# converting certain columns of DataFrame to float for ease later
cities_df[['Max Temperature (F)', 'Humidity', 'Cloudiness', 'Wind Speed (mph)']] = cities_df[['Max Temperature (F)', 'Humidity', 'Cloudiness', 'Wind Speed (mph)']].apply(pd.to_numeric)


# Displaying the dataframe
cities_df

,Latitude,Longitude,City,Country,Max Temperature (F),Humidity,Cloudiness,Wind Speed (mph),Date
0,-14.936039,13.322562,lubango,ao,69.80,56.0,0.0,3.44,1610528669
1,39.658409,-48.245995,torbay,ca,28.99,93.0,75.0,8.05,1610528850
2,-89.979413,37.550464,port elizabeth,za,71.60,83.0,90.0,13.80,1610528788
3,-35.580137,165.168906,ahipara,nz,65.88,84.0,10.0,6.15,1610528850
4,-63.583940,-38.583740,ushuaia,ar,41.00,75.0,0.0,8.05,1610528649
...,...,...,...,...,...,...,...,...,...
567,50.440424,-110.868744,medicine hat,ca,45.00,65.0,1.0,14.97,1610528977
568,8.361826,124.960650,maluko,ph,80.01,85.0,100.0,5.01,1610528978
569,56.959632,1.215939,boddam,gb,35.60,80.0,96.0,10.36,1610528978
570,16.191234,-76.291348,morant bay,jm,75.20,78.0,20.0,8.05,1610528978


In [14]:
# Exporting the dataframe to csv
cities_df.to_csv("city_weather_data.csv", encoding="utf-8", index=False)

In [ ]:
cities_df.dtypes

# because objects.

In [ ]:
cities_df.describe()

In [ ]:
# Checking if any humidity outliers by seeing if they are greater than the max

over100 = cities_df.loc[(cities_df["Humidity"] > 100)]
over100

In [ ]:
# There are none so it is okay to go to the plots with citities_df
# but dropping nan issues

cities_df = cities_df.dropna()

cities_df

# now 520 rows

In [ ]:
# Temperature (F) vs. Latitude
graph1 = cities_df.plot(kind = "scatter", x = "Latitude", y = "Max Temperature (F)", title = "Max Temperature (F) vs. Latitude")
graph1


In [ ]:
# Humidity (%) vs. Latitude
graph2 = cities_df.plot(kind = "scatter", x = "Latitude", y = "Humidity", title = "Humidity (%) vs. Latitude")
graph2

In [ ]:
# Cloudiness (%) vs. Latitude
graph3 = cities_df.plot(kind = "scatter", x = "Latitude", y = "Cloudiness", title = "Cloudiness (%) vs. Latitude")
graph3

In [ ]:
# Wind Speed (mph) vs. Latitude
graph4 = cities_df.plot(kind = "scatter", x = "Latitude", y = "Wind Speed (mph)", title = "Wind Speed (mph) vs. Latitude")
graph4

In [ ]:
# separating into hemispheres
#Northern Hemisphere (greater than or equal to 0 degrees latitude)  
#Southern Hemisphere (less than 0 degrees latitude)

northern_df = cities_df.loc[(cities_df["Latitude"] >= 0)]
northern_df


In [ ]:
southern_df = cities_df.loc[(cities_df["Latitude"] < 0)]
southern_df

In [ ]:
# Temperature (F) vs. Latitude
# Northern Hemisphere
graph1_north = northern_df.plot(kind = "scatter", x = "Latitude", y = "Max Temperature (F)", 
                                title = "Northern Hemishpere - Max Temperature (F) vs. Latitude Linear Regression")


# Perform a linear regression on temperature vs. latitude
# Calculating Regression line
x_value = northern_df["Latitude"]
y_value = northern_df["Max Temperature (F)"]

# so if i type stats.linregress I dont need to import it. importing stats is enough
(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_value, y_value)

# Get regression values
regress_values = x_value * slope + intercept

# Create line equation string
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Create Plot
#plt.scatter(x_value, y_value)

# Label plot and annotate the line equation
#plt.xlabel("Latitude")
#plt.ylabel("Temperature")
#plt.title("Temperature vs. Latitude")
plt.plot(x_value, regress_values, "r-")
plt.annotate(line_eq,(55,15), fontsize=12,color="red")


# Print r square value
print(f"The r squared value is: {round(rvalue, 4)}.")

# Show plot
plt.show()



In [ ]:
# Temperature (F) vs. Latitude
# SOUthern Hemisphere
graph1_south = southern_df.plot(kind = "scatter", x = "Latitude", y = "Max Temperature (F)", 
                                title = "Southern Hemishpere - Max Temperature (F) vs. Latitude Linear Regression")


# Perform a linear regression on temperature vs. latitude
# Calculating Regression line
x_value = southern_df["Latitude"]
y_value = southern_df["Max Temperature (F)"]

# so if i type stats.linregress I dont need to import it. importing stats is enough
(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_value, y_value)

# Get regression values
regress_values = x_value * slope + intercept

# Create line equation string
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Create Plot
#plt.scatter(x_value, y_value)

# Label plot and annotate the line equation
#plt.xlabel("Latitude")
#plt.ylabel("Temperature")
#plt.title("Temperature vs. Latitude")
plt.plot(x_value, regress_values, "r-")
plt.annotate(line_eq,(-80,10), fontsize=12,color="red")


# Print r square value
print(f"The r squared value is: {round(rvalue, 4)}.")

# Show plot
plt.show()


In [ ]:
# Humidity (%) vs. Latitude
# Northern Hemisphere
graph2_north = northern_df.plot(kind = "scatter", x = "Latitude", y = "Humidity", 
                                title = "Northern Hemishpere - Humidity (%) vs. Latitude Linear Regression")


# Perform a linear regression on temperature vs. latitude
# Calculating Regression line
x_value = northern_df["Latitude"]
y_value = northern_df["Humidity"]

# so if i type stats.linregress I dont need to import it. importing stats is enough
(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_value, y_value)

# Get regression values
regress_values = x_value * slope + intercept

# Create line equation string
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Create Plot
#plt.scatter(x_value, y_value)

# Label plot and annotate the line equation
#plt.xlabel("Latitude")
#plt.ylabel("Temperature")
#plt.title("Temperature vs. Latitude")
plt.plot(x_value, regress_values, "r-")
plt.annotate(line_eq,(55,15), fontsize=12,color="red")


# Print r square value
print(f"The r squared value is: {round(rvalue, 4)}.")

# Show plot
plt.show()



In [ ]:
# Humidity (%) vs. Latitude
# SOUthern Hemisphere
graph2_south = southern_df.plot(kind = "scatter", x = "Latitude", y = "Humidity", 
                                title = "Southern Hemishpere - Humidity (%) vs. Latitude Linear Regression")


# Perform a linear regression on temperature vs. latitude
# Calculating Regression line
x_value = southern_df["Latitude"]
y_value = southern_df["Humidity"]

# so if i type stats.linregress I dont need to import it. importing stats is enough
(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_value, y_value)

# Get regression values
regress_values = x_value * slope + intercept

# Create line equation string
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Create Plot
#plt.scatter(x_value, y_value)

# Label plot and annotate the line equation
#plt.xlabel("Latitude")
#plt.ylabel("Temperature")
#plt.title("Temperature vs. Latitude")
plt.plot(x_value, regress_values, "r-")
plt.annotate(line_eq,(-88,58), fontsize=12,color="red")


# Print r square value
print(f"The r squared value is: {round(rvalue, 4)}.")

# Show plot
plt.show()

In [ ]:
# Cloudiness (%) vs. Latitude
# Northern Hemisphere
graph3_north = northern_df.plot(kind = "scatter", x = "Latitude", y = "Cloudiness", 
                                title = "Northern Hemishpere - Cloudiness (%) vs. Latitude Linear Regression")


# Perform a linear regression on temperature vs. latitude
# Calculating Regression line
x_value = northern_df["Latitude"]
y_value = northern_df["Cloudiness"]

# so if i type stats.linregress I dont need to import it. importing stats is enough
(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_value, y_value)

# Get regression values
regress_values = x_value * slope + intercept

# Create line equation string
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Create Plot
#plt.scatter(x_value, y_value)

# Label plot and annotate the line equation
#plt.xlabel("Latitude")
#plt.ylabel("Temperature")
#plt.title("Temperature vs. Latitude")
plt.plot(x_value, regress_values, "r-")
plt.annotate(line_eq,(60,50), fontsize=12,color="red")

# Print r square value
print(f"The r squared value is: {round(rvalue, 4)}.")

# Show plot
plt.show()



In [ ]:
# Cloudiness (%) vs. Latitude
# Southern Hemisphere
graph3_south = southern_df.plot(kind = "scatter", x = "Latitude", y = "Cloudiness", 
                                title = "Southern Hemishpere - Cloudiness (%) vs. Latitude Linear Regression")


# Perform a linear regression on temperature vs. latitude
# Calculating Regression line
x_value = southern_df["Latitude"]
y_value = southern_df["Cloudiness"]

# so if i type stats.linregress I dont need to import it. importing stats is enough
(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_value, y_value)

# Get regression values
regress_values = x_value * slope + intercept

# Create line equation string
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Create Plot
#plt.scatter(x_value, y_value)

# Label plot and annotate the line equation
#plt.xlabel("Latitude")
#plt.ylabel("Temperature")
#plt.title("Temperature vs. Latitude")
plt.plot(x_value, regress_values, "r-")
plt.annotate(line_eq,(-90,53), fontsize=12,color="red")


# Print r square value
print(f"The r squared value is: {round(rvalue, 4)}.")

# Show plot
plt.show()

In [ ]:
# Wind Speed (mph) vs. Latitude
# Northern Hemisphere
graph4_north = northern_df.plot(kind = "scatter", x = "Latitude", y = "Wind Speed (mph)", 
                                title = "Northern Hemishpere - Wind Speed (mph) vs. Latitude Linear Regression")


# Perform a linear regression on temperature vs. latitude
# Calculating Regression line
x_value = northern_df["Latitude"]
y_value = northern_df["Wind Speed (mph)"]

# so if i type stats.linregress I dont need to import it. importing stats is enough
(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_value, y_value)

# Get regression values
regress_values = x_value * slope + intercept

# Create line equation string
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Create Plot
#plt.scatter(x_value, y_value)

# Label plot and annotate the line equation
#plt.xlabel("Latitude")
#plt.ylabel("Temperature")
#plt.title("Temperature vs. Latitude")
plt.plot(x_value, regress_values, "r-")
plt.annotate(line_eq,(40,12), fontsize=12,color="red")

# Print r square value
print(f"The r squared value is: {round(rvalue, 4)}.")

# Show plot
plt.show()

In [ ]:
# Wind Speed (mph) vs. Latitude
# Southern Hemisphere
graph4_south = southern_df.plot(kind = "scatter", x = "Latitude", y = "Wind Speed (mph)", 
                                title = "Southern Hemishpere - Wind Speed (mph) vs. Latitude Linear Regression")


# Perform a linear regression on temperature vs. latitude
# Calculating Regression line
x_value = southern_df["Latitude"]
y_value = southern_df["Wind Speed (mph)"]

# so if i type stats.linregress I dont need to import it. importing stats is enough
(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_value, y_value)

# Get regression values
regress_values = x_value * slope + intercept

# Create line equation string
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Create Plot
#plt.scatter(x_value, y_value)

# Label plot and annotate the line equation
#plt.xlabel("Latitude")
#plt.ylabel("Temperature")
#plt.title("Temperature vs. Latitude")
plt.plot(x_value, regress_values, "r-")
plt.annotate(line_eq,(-88,6), fontsize=12,color="red")


# Print r square value
print(f"The r squared value is: {round(rvalue, 4)}.")

# Show plot
plt.show()